# Task for Today  

***

## Credit Card Customer Attrition Prediction  

Given *data about credit card customers*, let's try to predict whether a given customer will **close their account**.  
  
We will use a few different models to make our predictions.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_csv('../input/credit-card-customers/BankChurners.csv')

In [ ]:
data

# Preprocessing

In [ ]:
data.isna().sum()

In [ ]:
data['Attrition_Flag'].unique()

In [ ]:
{column: list(data[column].unique()) for column in data.select_dtypes('object').columns}

In [ ]:
def binary_encode(df, column, positive_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def ordinal_encode(df, column, ordering):
    df = df.copy()
    df[column] = df[column].apply(lambda x: ordering.index(x))
    return df

def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop last two columns (unneeded)
    df = df.drop(df.columns[-2:], axis=1)
    
    # Drop CLIENTNUM columns
    df = df.drop('CLIENTNUM', axis=1)
    
    # Encode unknown values as np.NaN
    df = df.replace('Unknown', np.NaN)
    
    # Fill ordinal missing values with modes (Education_Level and Income_Category columns)
    df['Education_Level'] = df['Education_Level'].fillna('Graduate')
    df['Income_Category'] = df['Income_Category'].fillna('Less than $40K')
    
    # Encode binary columns
    df = binary_encode(df, 'Attrition_Flag', positive_value='Attrited Customer')
    df = binary_encode(df, 'Gender', positive_value='M')
    
    # Encode ordinal columns
    education_ordering = [
        'Uneducated',
        'High School',
        'College',
        'Graduate',
        'Post-Graduate',
        'Doctorate'
    ]
    income_ordering = [
        'Less than $40K',
        '$40K - $60K',
        '$60K - $80K',
        '$80K - $120K',
        '$120K +'
    ]
    df = ordinal_encode(df, 'Education_Level', ordering=education_ordering)
    df = ordinal_encode(df, 'Income_Category', ordering=income_ordering)
    
    # Encode nominal columns
    df = onehot_encode(df, 'Marital_Status', prefix='MS')
    df = onehot_encode(df, 'Card_Category', prefix='CC')
    
    # Split df into X and y
    y = df['Attrition_Flag'].copy()
    X = df.drop('Attrition_Flag', axis=1).copy()
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    return X, y

In [ ]:
X, y = preprocess_inputs(data)

In [ ]:
X

In [ ]:
y

# Exploratory Data Analysis

In [ ]:
# Excluding categorical columns
eda_df = pd.concat([X.loc[:, ['Customer_Age', 'Months_on_book']], X.loc[:,'Credit_Limit':'Avg_Utilization_Ratio']], axis=1).copy()

In [ ]:
eda_df

## Univariate Visualization

In [ ]:
plt.figure(figsize=(24, 15))

for i in range(len(eda_df.columns) - 1):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(eda_df[eda_df.columns[i]])

plt.show()

In [ ]:
plt.figure(figsize=(24, 15))

for i in range(len(eda_df.columns) - 1):
    plt.subplot(3, 3, i + 1)
    sns.distplot(eda_df[eda_df.columns[i]])

plt.show()

## Multivariate Analysis

In [ ]:
plt.figure(figsize=(20, 20))
sns.pairplot(eda_df)
plt.show()

In [ ]:
corr = pd.concat([eda_df, y], axis=1).corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, vmin=-1.0, cmap='mako')
plt.show()

# Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [ ]:
models = [
    LogisticRegression(),
    SVC(),
    DecisionTreeClassifier(),
    MLPClassifier(),
    RandomForestClassifier()
]

for model in models:
    model.fit(X_train, y_train)

In [ ]:
model_names = [
    "   Logistic Regression",
    "Support Vector Machine",
    "         Decision Tree",
    "        Neural Network",
    "         Random Forest"
]

for model, name in zip(models, model_names):
    print(name + ": {:.4f}%".format(model.score(X_test, y_test) * 100))

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/pyeUcmmLeiM